# Calculating yields and prediction of expression modulation targets

The following chapter deals with the calculation of product and biomass yields in relation to different carbon sources and the prediction of expression modulation targets.

In [ ]:
!pip install cobra

In [ ]:
import os
from cobra.io import read_sbml_model

if not os.path.isfile('e_coli_core.xml.gz'): #checking whether model file exist already
    !wget http://bigg.ucsd.edu/static/models/e_coli_core.xml.gz

model_original = read_sbml_model('e_coli_core.xml.gz')

## Product yield

To preserve the original model, a copy of the model is first created for the changes.

In [ ]:
model = model_original.copy()

Calculating the yield of a product is straigtforward. First one sets the producing reaction a new objective of the model.

In [ ]:
model.objective = model.reactions.EX_ac_e

Then one determines the maximum production flux by solving the model.

In [ ]:
solution = model.optimize()
solution.fluxes['EX_ac_e']

Dividing the production flux by the uptake flux of the carbon source (in this case glucose) yields the molar yield (mol product / mol glucose).

In [ ]:
solution.fluxes['EX_ac_e'] / (-1. * solution.fluxes['EX_glc__D_e'])

## Exercise (15 min)
Convert the yield into a C-mol (product) / C-mol (carbon source) yield.

Calculate a mass yield instead of molar yield (g (product) / g (carbon source))

## Biomass Yield

The ratio of the amount of biomass produced to the amount of substrate consumed (g biomass/g substrate) is defined as the biomass yield, and typically is defined relative to the electron donor used.


In [ ]:
model = model_original.copy()

In [ ]:
solution = model.optimize()
g_dw_biomass_per_mmol_glucose = solution.objective_value / (-1. * solution.fluxes['EX_glc__D_e'])
g_dw_biomass_per_mmol_glucose

## Excercise (10 min)
The units of this yield are gDW (Biomass) / mmol (glucose). Convert this number into a mass yield.

## Growth vs. Product yield
## Exercise (30 min)
In the previous section we calculated theoretical maximum yields in the absence of competing objectives. But how does product yield vary with growth for example?

Determine molar, mass, and c-mol yields of acetate for the following growth rates (plot them using the code snippet below). How do the yields looks like for anaerobic conditions?

In [ ]:
model = model_original.copy()

In [ ]:
growth_rates = [0.0, 0.10915242, 0.21830485, 0.32745727, 0.43660969, 0.54576212, 0.65491454, 0.76406697, 0.87321939, 0.98237181]

Fill in your own product fluxes here (needs to be a list).

In [ ]:
pip install matplotlib

In [ ]:
product_fluxes = [0] * len(growth_rates) # replace this
import matplotlib.pyplot as plt
plt.plot(growth_rates, product_fluxes)
plt.xlabel('Growth [$h^{-1}$]')
plt.ylabel('Product flux [$mmol gDW^{-1} h^{-1}$]')
plt.show()

Fill in your own product yields here (needs to be a list).

In [ ]:
yields = [0] * len(growth_rates) # replace this
plt.plot(growth_rates, yields)
plt.xlabel('Growth [$h^{-1}$]')
plt.ylabel('Product yield [$mmol(acetate) / mmol(glucose)$]')
plt.show()

## Predict expression modulation targets

Cameo provides algorithms to search for genes or reactions that can be over or down regulated in order to achieve a given biological objective. Plotly's Python graphing library makes interactive, publication-quality graphs.

## Succinate production
The production envelope looks like this.

You have to install cameo. Cameo is a high-level python library developed to aid the strain design process in metabolic engineering projects. The library provides a modular framework of simulation and strain design methods that targets developers that want to develop new design algorithms and custom analysis workflows. 

In [ ]:
pip install cameo

Plotly's Python graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.

pip install plotly

In [ ]:
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
plotter = PlotlyPlotter()
production_envelope = phenotypic_phase_plane(model, 
                                             variables=[model.reactions.BIOMASS_Ecoli_core_w_GAM],
                                             objective=model.metabolites.succ_e)
production_envelope.plot(plotter, height=400)

## Flux Scanning based on Enforced Objective Flux (FSEOF)

Concept of FSEOF framework to select the gene amplification targets for enhanced product formation. FSEOF searches for the candidate fluxes to be amplified through scanning for those fluxes that increase with enforced objective (product formation) flux under the objective function of maximizing biomass formation flux. During the FSEOF implementation, three types of intracellular flux profiles are typically identified, increased, decreased, and unchanged; oscillating flux profiles can be found in rare cases. Among them, FSEOF identifies the fluxes showing the increased profile as the primary amplification targets.

In [ ]:
from cameo.strain_design.deterministic.flux_variability_based import FSEOF

In [ ]:
fseof = FSEOF(model)

In [ ]:
result = fseof.run(target=model.reactions.EX_succ_e)

In [ ]:
result.data_frame

The results of the analysis can be displayed graphically using Matplotlib. Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

In [ ]:
from matplotlib import pyplot as plt
result.data_frame.T.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()